In [3]:
spec = """
<b># Meteo Web App: Real-Time Weather & Clothing Advice</b>

## Overview
This web application provides users with real-time weather information for any location, along with personalized clothing recommendations based on the current weather conditions.

## Features

### Real-Time Weather Display
- Interactive map showing current weather conditions globally
- Temperature, humidity, wind speed, and precipitation probability
- Visual representation of weather (sun, clouds, rain, snow)
- Real-time updates every 5 minutes

### Location Search
- Search for any city, address, or geographic coordinate
- Autocomplete suggestions as you type
- Recent searches and favorites list
- Current location detection with GPS

### Clothing Advice
- Personalized clothing recommendations based on current temperature, humidity, and weather conditions
- Layering suggestions appropriate for weather conditions
- Clothing material recommendations for comfort and appropriateness
- Reminders for appropriate footwear for wet or snowy conditions

### Additional Features
- 7-day forecast
- Weather alerts for extreme conditions
- Air quality index
- Sunrise/sunset times

## User Experience
- Clean, intuitive interface
- Responsive design for all devices
- Accessible for users with disabilities
- Multi-language support

## Technical Implementation

### Frontend
- React.js or Vue.js for dynamic user interface
- Leaflet or Mapbox for interactive maps
- Weather icons from OpenWeather
- Responsive CSS framework (Bootstrap or Tailwind)

### Backend
- Node.js/Express or Python/Flask for API
- PostgreSQL or MongoDB for data storage
- RESTful API design

### External APIs
- OpenWeather API for current and forecast data
- News API for weather-related news (optional)
- Geolocation services for IP-based location

## Development Plan

### Phase 1: Setup and API Integration
- Register for OpenWeather API key
- Set up basic web app structure
- Integrate OpenWeather API
- Implement location search functionality

### Phase 2: Weather Display and User Interface
- Develop real-time weather display
- Create interactive map
- Implement temperature and weather condition visualization
- Add clothing advice module

### Phase 3: Additional Features and Testing
- Develop 7-day forecast
- Implement alerts and air quality
- Test all features thoroughly
- Optimize performance and loading times

### Phase 4: Launch and Marketing
- Deploy to production
- Implement analytics
- Launch marketing campaign
- Gather user feedback and iterate

## Conclusion
This Meteo web app will provide users with a comprehensive and personalized weather experience, helping them make informed decisions about their daily activities and clothing choices. By leveraging the OpenWeather API and focusing on a clean, intuitive user interface, we can create a valuable tool for anyone needing up-to-date weather information.
"""

In [ ]:
from pyagentspec.agent import Agent
from pyagentspec.llms import OpenAiConfig
import os
from pyagentspec.tools import ServerTool
from pyagentspec.property import StringProperty

GEMINI_MODEL_ID = "gemini-2.5-flash"
GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
from pyagentspec.llms.openaicompatibleconfig import OpenAiCompatibleConfig

llm_config = OpenAiCompatibleConfig(
    name="gemini-llm",
    model_id=GEMINI_MODEL_ID,
    url=GEMINI_URL
)

create_folder_tool = ServerTool(
    name="create_folder",  # this is the name the LLM will see and call
    description="Create a folder with the given name.",
    inputs=[StringProperty(title="name")],
)

write_file_tool = ServerTool(
    name="write_file",
    description="Write header functions to a file given the project specification.",
    inputs=[StringProperty(title="path"), StringProperty(title="text")],
)



def create_folder(name: str) -> str:
    os.makedirs(name, exist_ok=True)
    return f"Folder '{name}' created."

def create_file(name: str) -> str:
    with open(name, 'w') as f:
        f.write("")  # create an empty file
    return f"File '{name}' created."


def write_file(path: str, text: str) -> None:
    with open(path, 'w') as file:
        file.write(text)
        file.close()
def rewrite_file(path: str, new_content: str) -> str:
    """
    Overwrite an existing file with new content **after** the LLM analyzes
    the current file via read_file().

    Args:
        path (str): Path to the file that should be rewritten.
        new_content (str): Full rewritten content to replace the original.

    Returns:
        str: Description of the update performed.

    Raises:
        FileNotFoundError: If the file does not exist.
        IsADirectoryError: If the path refers to a directory.
        Exception: If writing fails.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"File '{path}' does not exist.")

    if os.path.isdir(path):
        raise IsADirectoryError(f"'{path}' is a directory, not a file.")

    with open(path, "w", encoding="utf-8") as f:
        f.write(new_content)

    return f"Rewrote entire file '{path}'."

def read_file(path: str) -> str:
    """
    Read and return the content of a text file.

    Args:
        path (str): Path to the file.

    Returns:
        str: File content as a string.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"File '{path}' does not exist.")
    if os.path.isdir(path):
        raise IsADirectoryError(f"'{path}' is a directory, not a file.")

    with open(path, "r", encoding="utf-8") as f:
        return f.read()
    

create_file_tool = ServerTool(
    name="create_file",  # this is the name the LLM will see and call
    description="Create a file with the given name.",
    inputs=[StringProperty(title="name")],
)
create_folder_tool = ServerTool(
    name="create_folder",  # this is the name the LLM will see and call
    description="Create a folder with the given name.",
    inputs=[StringProperty(title="name")],
)
rewrite_file_tool = ServerTool(
    name="rewrite_file",
    description="Overwrite an existing code file with new content.",
    inputs=[
        StringProperty(title="path"),
        StringProperty(title="new_content")
    ],
)

read_file_tool = ServerTool(
    name="read_file",
    description="Read and return the content of a source file at the given path.",
    inputs=[StringProperty(title="path")],
)

prompt = """
You are given a natural-language specification of a software project.

Your task is to:

1. Create a single top-level directory named after the application described in the specification.

2. Inside this directory, infer a complete project directory structure (you may use subfolders such as frontend, backend, services, utils, tests, etc.).

3. If the specification describes a web application (e.g., a site or web app with a browser-based UI):
   - Scaffold a minimal but functional web application inside the top-level directory.
   - Choose a reasonable tech stack based on the specification (for example: a React/Vite/Next.js/Vanilla JS frontend, and optionally a Node/Express or Python backend).
   - Create all necessary configuration files to run the app (for example: `package.json`, build config, basic entrypoints like `src/main.jsx`, `public/index.html`, or similar).
   - Ensure that after dependencies are installed, the app can be started with a standard command (such as `npm start`, `npm run dev`, or the equivalent for the chosen stack).

4. Propose concrete file names for each folder.

5. For every file, provide a short description of its purpose.

6. For every file, list the key functions it should contain, each defined by:
   - "name": the function name
   - "args": an array of argument names
   - "description": a short explanation of what the function does

7. **Physically create** the entire directory structure and all files using your available tools.

8. Inside each file:
   - Implement the functions

9. After creation, return the final project structure as a single JSON object following the example format below.

Example format (do NOT reuse the example names, files, or functions):

MainProject
{
  "frontend": {
    "MainProject/frontend/index.html": "HTML structure for the homepage",
    "MainProject/frontend/app.js": "Main JavaScript entry point",
    "functions": [
      {
        "name": "renderHomepage",
        "args": ["data"],
        "description": "Renders the homepage using provided data."
      }
    ]
  },
  "backend": {
    "MainProjectbackend/server.py": "Flask server handling routes",
    "functions": [
      {
        "name": "start_server",
        "args": ["host", "port"],
        "description": "Launches the server on the given host and port."
      }
    ]
  }
}

Requirements and constraints:
- Once you generated code, you should read it, rewrite it if it needs improvement
- All generated folders and files must be placed under a single top-level directory named after the application.
- Subfolders are allowed  (e.g., frontend/components, backend/services).
- Each folder value in the JSON output must be a JSON object.
- Inside each folder:
  - Files must be represented as:
        "path/to/fileName.ext": "Short description"
  - A "functions" key may describe the functions contained in those files.
- The agent must **actually create** all directories and files described.
- If the project is a web application, the agent must also create a runnable web app scaffold (including any required configuration/entry files).
- Output valid JSON with double quotes and no trailing commas
-Be sure that the code is of quality, remember created function to reuse them later.
"""


agent2 = Agent(
    name="Structure Creation Agent",
    llm_config=llm_config,
    tools=[create_folder_tool, create_file_tool, write_file_tool,rewrite_file_tool, read_file_tool], 
    system_prompt=(prompt),
)

from pyagentspec.serialization import AgentSpecSerializer
from wayflowcore.agentspec import AgentSpecLoader
from wayflowcore.executors.executionstatus import UserMessageRequestStatus
from dotenv import load_dotenv

load_dotenv()
# 3) Serialize to JSON (framework-agnostic spec)
serialized_agent2 = AgentSpecSerializer().to_json(agent2)



tool_registry = {
    "create_folder": create_folder,
    "create_file": create_file,
    "write_file": write_file,
    "rewrite_file" :rewrite_file,
    "read_file" : read_file
}

loader1 = AgentSpecLoader()
loader2 = AgentSpecLoader(tool_registry=tool_registry)
wayflow_agent2 = loader2.load_json(serialized_agent2) 



conversation = wayflow_agent2.start_conversation()
conversation.append_user_message(spec)
conversation.execute()
messages = conversation.get_messages()


NameError: name 'spec' is not defined